In [1]:
import sys, os

In [2]:
sys.path.append('src')

In [4]:
import pickle

with open('../src/models/stam_model.pkl','rb') as read_file:
    stam_model = pickle.load(read_file)
    
with open('../src/models/tech_model.pkl', 'rb') as read_file:
    tech_model = pickle.load(read_file)

In [23]:
features = ['song_seconds', 'step_count', 'measure_count', 'bpm_weighted_avg',
       'bpm_max', 'bpm_min', 'bpm_mode', 'bpm_change_count', 'song_nps',
       'nps_per_measure_max', 'nps_per_measure_avg', 'nps_per_measure_median',
       'nps_per_measure_std', 'nps_per_measure_mode', 'jumps', 'hands',
       'quads', 'holds', 'mines', 'rolls', 'crossovers', 'footswitches',
       'crossover_footswitches', 'jacks', 'invalid_crossovers', 'stop_count',
       'stream_total', 'stream_count', 'stream_size_max', 'stream_size_avg',
       'stream_size_std', 'break_count', 'break_size_max', 'break_size_avg',
       'break_total', 'break_size_std', 'stream_log_transform']

stam_features_dict = {}
for feature in features:
    stam_features_dict[feature] = [0]    
    
stam_features_dict['step_count'] = 1400
stam_features_dict['song_nps'] = 5.73
stam_features_dict['stream_total'] = 60

In [24]:
import pandas as pd

X = pd.DataFrame.from_dict(stam_features_dict)

In [25]:
stam_model.predict(X)

array([4.52302], dtype=float32)

In [26]:
features = ['song_seconds', 'step_count', 'measure_count', 'bpm_weighted_avg',
       'bpm_max', 'bpm_min', 'bpm_mode', 'bpm_change_count', 'song_nps',
       'nps_per_measure_max', 'nps_per_measure_avg', 'nps_per_measure_median',
       'nps_per_measure_std', 'nps_per_measure_mode', 'jumps', 'hands',
       'quads', 'holds', 'mines', 'rolls', 'crossovers', 'footswitches',
       'crossover_footswitches', 'jacks', 'invalid_crossovers', 'stop_count',
       'stream_total', 'stream_log_transform']

tech_features = {}
for feature in features:
    tech_features[feature] = [0]

tech_features['step_count'] = 1400
tech_features['song_nps'] = 5.73
tech_features['stream_total'] = 60

In [27]:
X_tech = pd.DataFrame.from_dict(tech_features)

In [28]:
tech_model.predict(X_tech)

array([5.9540114], dtype=float32)

In [59]:
data = dict({'step_count': 1400, 'song_nps': 5.73, 'stream_total': 60, 'datatype': 1})
data

{'step_count': 1400, 'song_nps': 5.73, 'stream_total': 60, 'datatype': 1}

In [29]:
def extract_feature_values(data):
    """ Given a params dict, return the values for feeding into a model"""
    
    # Replace these features with the features for your model. They need to 
    # correspond with the `name` attributes of the <input> tags

    #set stamina = none by default to use the tech predictor, change to true if expected input 'datatype' = 1
    stamina = None

    #expected input ["step_count", "song_nps", "stream_total", "datatype"]

    STAM_FEATURES = ['song_seconds', 'step_count', 'measure_count', 'bpm_weighted_avg',
       'bpm_max', 'bpm_min', 'bpm_mode', 'bpm_change_count', 'song_nps',
       'nps_per_measure_max', 'nps_per_measure_avg', 'nps_per_measure_median',
       'nps_per_measure_std', 'nps_per_measure_mode', 'jumps', 'hands',
       'quads', 'holds', 'mines', 'rolls', 'crossovers', 'footswitches',
       'crossover_footswitches', 'jacks', 'invalid_crossovers', 'stop_count',
       'stream_total', 'stream_count', 'stream_size_max', 'stream_size_avg',
       'stream_size_std', 'break_count', 'break_size_max', 'break_size_avg',
       'break_total', 'break_size_std', 'stream_log_transform']

    TECH_FEATURES = ['song_seconds', 'step_count', 'measure_count', 'bpm_weighted_avg',
       'bpm_max', 'bpm_min', 'bpm_mode', 'bpm_change_count', 'song_nps',
       'nps_per_measure_max', 'nps_per_measure_avg', 'nps_per_measure_median',
       'nps_per_measure_std', 'nps_per_measure_mode', 'jumps', 'hands',
       'quads', 'holds', 'mines', 'rolls', 'crossovers', 'footswitches',
       'crossover_footswitches', 'jacks', 'invalid_crossovers', 'stop_count',
       'stream_total', 'stream_log_transform']

    # if expected input is 0, this data is tech data and will be processed using the TECH_FEATURES list
    # stamina Boolean value will be set to false
    if data["datatype"] == 0:
        stamina = False
        lis = list(map(data.get, TECH_FEATURES))
        values_list = [0 if val is None else val for val in lis]
        
        return pd.DataFrame(data = [values_list], columns=TECH_FEATURES), stamina
    
    
    else:
        stamina = True 
        lis = list(map(data.get, STAM_FEATURES))
        values_list = [0 if val is None else val for val in lis]
        
        return pd.DataFrame(data = [values_list], columns=STAM_FEATURES), stamina

In [60]:
feature_values, stamina = extract_feature_values(data)
stamina

True

In [54]:
def get_prediction(feature_values, stamina):
    """ Given a list of feature values, return a prediction made by the model"""
    
    stam_model, tech_model = un_pickle_model()
    
    #Given a boolean for whether the data is stamina data or not, predicts using one of the two models
    if stamina:
        predictions = stam_model.predict(feature_values)
        return predictions[0]
    # We are only making a single prediction, so return the 0-th value
    elif stamina == False:
        predictions = tech_model.predict(feature_values)
        return predictions[0]

def un_pickle_model():
    """ Load the model from the .pkl file """
    with open('../src/models/stam_model.pkl','rb') as stam_file:
        stam = pickle.load(stam_file)
    
    with open('../src/models/tech_model.pkl', 'rb') as tech_file:
        tech = pickle.load(tech_file)

    return stam, tech

In [52]:
stam_model, tech_model = un_pickle_model()

In [61]:
get_prediction(feature_values, stamina)

4.52302